In [164]:
import pandas as pd
import numpy as np
import json
import requests

In [165]:
games = pd.read_csv(
        '/Users/mattbubb/Spring_2021/DS_340W/Sports-Betting-340W/Data/nba_games_all.csv')
#colToDrop = ['oreb', 'dreb', 'reb', 'ast',
                 #'stl', 'blk', 'tov', 'pf', 'pts', 'min', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct']
g = games
moneyLine = pd.read_csv(
        '/Users/mattbubb/Spring_2021/DS_340W/Sports-Betting-340W/Data/nba_betting_money_line.csv')


In [166]:
moneyLine = pd.read_csv(
        '/Users/mattbubb/Spring_2021/DS_340W/Sports-Betting-340W/Data/nba_betting_money_line.csv')
g = g.sort_values('game_date', ascending=False).merge(moneyLine, on='game_id').dropna()


In [167]:
# TAKES ABOUT 5 MIN TO RUN

rowCount = 0
gameCount = 0
g['joinOn'] = 0
for i in range(g.shape[0]):
    if rowCount < 2:
        g.loc[i,'joinOn'] = gameCount
        rowCount += 1
    else:
        gameCount += 1
        g.loc[i,'joinOn'] = gameCount
        rowCount = 1
g = g.dropna()


In [168]:
g.columns
a = g

a.head(50)

,game_id,game_date,matchup,team_id_x,is_home,wl,w,l,w_pct,min,...,season_year,season_type,season,book_name,book_id,team_id_y,a_team_id_y,price1,price2,joinOn
0,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-166.0,150.0,0.0
1,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,5Dimes,19.0,1.610613e+09,1.610613e+09,-11450.0,6725.0,0.0
2,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,Bookmaker,93.0,1.610613e+09,1.610613e+09,-180.0,155.0,1.0
3,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,BetOnline,1096.0,1.610613e+09,1.610613e+09,-167.0,147.0,1.0
4,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,Bovada,999996.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0
5,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,Heritage,169.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0
6,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,Intertops,180.0,1.610613e+09,1.610613e+09,-165.0,145.0,3.0
7,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,YouWager,139.0,1.610613e+09,1.610613e+09,-165.0,145.0,3.0
8,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,-180.0,155.0,4.0
9,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017.0,Playoffs,2017-18,Sportsbetting,999991.0,1.610613e+09,1.610613e+09,-167.0,147.0,4.0


In [169]:

a.loc[(a['is_home'] == 't') & (a['team_id_x'] == a['team_id_y']), 'homeTeamPrice'] = a['price1']
a.loc[(a['is_home'] == 't') & (a['team_id_x'] != a['team_id_y']), 'homeTeamPrice'] = a['price2']
a.loc[(a['is_home'] == 't') , 'awayTeamPrice'] = (a['price2'] + a['price1']) - a['homeTeamPrice']


a.loc[(a['is_home'] == 'f') & (a['team_id_x'] == a['team_id_y']), 'awayTeamPrice'] = a['price1']
a.loc[(a['is_home'] == 'f') & (a['team_id_x'] != a['team_id_y']), 'awayTeamPrice'] = a['price2']
a.loc[(a['is_home'] == 'f'), 'homeTeamPrice'] = (a['price2'] + a['price1']) - a['awayTeamPrice']





a.head(10)




,game_id,game_date,matchup,team_id_x,is_home,wl,w,l,w_pct,min,...,season,book_name,book_id,team_id_y,a_team_id_y,price1,price2,joinOn,homeTeamPrice,awayTeamPrice
0,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-166.0,150.0,0.0,150.0,-166.0
1,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,5Dimes,19.0,1.610613e+09,1.610613e+09,-11450.0,6725.0,0.0,6725.0,-11450.0
2,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Bookmaker,93.0,1.610613e+09,1.610613e+09,-180.0,155.0,1.0,155.0,-180.0
3,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,BetOnline,1096.0,1.610613e+09,1.610613e+09,-167.0,147.0,1.0,147.0,-167.0
4,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Bovada,999996.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0,150.0,-170.0
5,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Heritage,169.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0,150.0,-170.0
6,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Intertops,180.0,1.610613e+09,1.610613e+09,-165.0,145.0,3.0,145.0,-165.0
7,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,YouWager,139.0,1.610613e+09,1.610613e+09,-165.0,145.0,3.0,145.0,-165.0
8,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,-180.0,155.0,4.0,155.0,-180.0
9,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Sportsbetting,999991.0,1.610613e+09,1.610613e+09,-167.0,147.0,4.0,147.0,-167.0


In [170]:
### Make sure that every game is correctly identified by last column
droppedNA = a.dropna()
droppedNA

,game_id,game_date,matchup,team_id_x,is_home,wl,w,l,w_pct,min,...,season,book_name,book_id,team_id_y,a_team_id_y,price1,price2,joinOn,homeTeamPrice,awayTeamPrice
0,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-166.0,150.0,0.0,150.0,-166.0
1,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,5Dimes,19.0,1.610613e+09,1.610613e+09,-11450.0,6725.0,0.0,6725.0,-11450.0
2,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Bookmaker,93.0,1.610613e+09,1.610613e+09,-180.0,155.0,1.0,155.0,-180.0
3,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,BetOnline,1096.0,1.610613e+09,1.610613e+09,-167.0,147.0,1.0,147.0,-167.0
4,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Bovada,999996.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0,150.0,-170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250567,20600003.0,2006-11-01,ATL @ PHI,1.610613e+09,t,W,1.0,0.0,1.0,240.0,...,2006-07,Bovada,999996.0,1.610613e+09,1.610613e+09,200.0,-240.0,0.0,-240.0,200.0
250568,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,2006-07,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,215.0,-235.0,0.0,-235.0,215.0
250569,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,205.0,-245.0,0.0,-245.0,205.0
250570,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,2006-07,Bookmaker,93.0,1.610613e+09,1.610613e+09,200.0,-240.0,0.0,-240.0,200.0


In [171]:
# Take every other row starting on row 0
home  = g.iloc[0::2, :]
lst = []
for i in home.columns:
    lst.append("team1_"+i)
home.columns = lst
home = home.dropna().rename(columns = {'team1_joinOn':'joinOn'})
home

,team1_game_id,team1_game_date,team1_matchup,team1_team_id_x,team1_is_home,team1_wl,team1_w,team1_l,team1_w_pct,team1_min,...,team1_season,team1_book_name,team1_book_id,team1_team_id_y,team1_a_team_id_y,team1_price1,team1_price2,joinOn,team1_homeTeamPrice,team1_awayTeamPrice
0,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,-166.0,150.0,0.0,150.0,-166.0
2,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Bookmaker,93.0,1.610613e+09,1.610613e+09,-180.0,155.0,1.0,155.0,-180.0
4,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Bovada,999996.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0,150.0,-170.0
6,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Intertops,180.0,1.610613e+09,1.610613e+09,-165.0,145.0,3.0,145.0,-165.0
8,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,JustBet,1275.0,1.610613e+09,1.610613e+09,-180.0,155.0,4.0,155.0,-180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250562,20600005.0,2006-11-01,ORL vs. CHI,1.610613e+09,f,L,1.0,1.0,0.5,240.0,...,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,-125.0,105.0,0.0,105.0,-125.0
250564,20600003.0,2006-11-01,ATL @ PHI,1.610613e+09,t,W,1.0,0.0,1.0,240.0,...,2006-07,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,215.0,-235.0,0.0,-235.0,215.0
250566,20600003.0,2006-11-01,ATL @ PHI,1.610613e+09,t,W,1.0,0.0,1.0,240.0,...,2006-07,Bookmaker,93.0,1.610613e+09,1.610613e+09,200.0,-240.0,0.0,-240.0,200.0
250568,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,2006-07,Pinnacle Sports,238.0,1.610613e+09,1.610613e+09,215.0,-235.0,0.0,-235.0,215.0


In [178]:
# Take every other row starting on row 1
away = g.iloc[1::2, :]
lst = []
for i in away.columns:
    lst.append("team2_"+i)
away.columns = lst
away = away.dropna().rename(columns = {'team2_joinOn':'joinOn'})
away

,team2_game_id,team2_game_date,team2_matchup,team2_team_id_x,team2_is_home,team2_wl,team2_w,team2_l,team2_w_pct,team2_min,...,team2_season,team2_book_name,team2_book_id,team2_team_id_y,team2_a_team_id_y,team2_price1,team2_price2,joinOn,team2_homeTeamPrice,team2_awayTeamPrice
1,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,5Dimes,19.0,1.610613e+09,1.610613e+09,-11450.0,6725.0,0.0,6725.0,-11450.0
3,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,BetOnline,1096.0,1.610613e+09,1.610613e+09,-167.0,147.0,1.0,147.0,-167.0
5,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Heritage,169.0,1.610613e+09,1.610613e+09,-170.0,150.0,2.0,150.0,-170.0
7,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,YouWager,139.0,1.610613e+09,1.610613e+09,-165.0,145.0,3.0,145.0,-165.0
9,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,2017-18,Sportsbetting,999991.0,1.610613e+09,1.610613e+09,-167.0,147.0,4.0,147.0,-167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250563,20600005.0,2006-11-01,ORL vs. CHI,1.610613e+09,f,L,1.0,1.0,0.5,240.0,...,2006-07,Bookmaker,93.0,1.610613e+09,1.610613e+09,-120.0,100.0,0.0,100.0,-120.0
250565,20600003.0,2006-11-01,ATL @ PHI,1.610613e+09,t,W,1.0,0.0,1.0,240.0,...,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,205.0,-245.0,0.0,-245.0,205.0
250567,20600003.0,2006-11-01,ATL @ PHI,1.610613e+09,t,W,1.0,0.0,1.0,240.0,...,2006-07,Bovada,999996.0,1.610613e+09,1.610613e+09,200.0,-240.0,0.0,-240.0,200.0
250569,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,205.0,-245.0,0.0,-245.0,205.0


In [199]:
#How many total games 
masterDf = home.merge(away, how = 'left', on = 'joinOn')
#len(np.unique(masterDf['team1_game_id']))
#masterDf['label_prices'] = masterDf['team2_price1'].astype(str) +', '+ masterDf['team2_price2'].astype(str)
colDrop = ['team2_price1', 'team2_price2', 'team1_book_name', 'team1_book_id', 'team2_season']
masterDf.drop(colDrop, axis = 1)
masterDf

,team1_game_id,team1_game_date,team1_matchup,team1_team_id_x,team1_is_home,team1_wl,team1_w,team1_l,team1_w_pct,team1_min,...,team2_season_type,team2_season,team2_book_name,team2_book_id,team2_team_id_y,team2_a_team_id_y,team2_price1,team2_price2,team2_homeTeamPrice,team2_awayTeamPrice
0,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,Playoffs,2017-18,5Dimes,19.0,1.610613e+09,1.610613e+09,-11450.0,6725.0,6725.0,-11450.0
1,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,Regular Season,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,125.0,-145.0,-145.0,125.0
2,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,Regular Season,2006-07,Bovada,999996.0,1.610613e+09,1.610613e+09,125.0,-145.0,-145.0,125.0
3,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,Regular Season,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,125.0,-145.0,-145.0,125.0
4,41700404.0,2018-06-08,CLE vs. GSW,1.610613e+09,f,W,4.0,0.0,1.0,240.0,...,Regular Season,2006-07,Bovada,999996.0,1.610613e+09,1.610613e+09,125.0,-145.0,-145.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127682,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,Regular Season,2006-07,Bookmaker,93.0,1.610613e+09,1.610613e+09,-120.0,100.0,100.0,-120.0
127683,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,Regular Season,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,205.0,-245.0,-245.0,205.0
127684,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,Regular Season,2006-07,Bovada,999996.0,1.610613e+09,1.610613e+09,200.0,-240.0,-240.0,200.0
127685,20600003.0,2006-11-01,PHI vs. ATL,1.610613e+09,f,L,0.0,1.0,0.0,240.0,...,Regular Season,2006-07,5Dimes,19.0,1.610613e+09,1.610613e+09,205.0,-245.0,-245.0,205.0


In [217]:
# To do 
#masterDf.columns

# Drop duplicate columns
colsToDrop = ['team1_game_id', 'team2_game_id','team1_game_date','team1_min','team1_game_date','team1_a_team_id_x','team1_book_name',
              'team2_game_date', 'team2_min', 'team2_game_date', 'team2_a_team_id_x', 
             'team2_game_id','team2_book_name','team2_season_year','team2_matchup',
              'team1_wl','team2_wl','team1_w', 'team1_l','team2_w', 'team2_l', 'joinOn',
             'team1_homeTeamPrice', 'team1_awayTeamPrice','team1_team_id_y',
             'team1_a_team_id_y','team2_season_type', 'team1_season_year', 
              'team2_is_home','team2_pf', 'team2_pts', 'team2_season',
             'team2_team_id_y', 'team2_a_team_id_y', 'team2_price1',
             'team2_price2','team1_book_id', 'team1_price1', 'team1_price2','team1_pf',
              'team1_pts', 'team1_season_type','team1_season','team1_is_home','team1_fga','team1_fg3m', 'team1_fg3a',
             'team1_oreb','team1_dreb','team1_ftm','team2_fga','team2_fg3m', 'team2_fg3a','team2_oreb', 'team2_dreb'
             ]
predict = masterDf.drop(colsToDrop, axis = 1)
predict = predict.rename({"team2_book_id" : "book_id","team2_homeTeamPrice": "homeTeamPrice", "team2_awayTeamPrice": "awayTeamPrice" }, axis = 1)
quantitative = ['team1_w_pct', 'team1_fgm',
       'team1_fg_pct', 'team1_fg3_pct', 'team1_fta', 'team1_ft_pct',
       'team1_reb', 'team1_ast', 'team1_stl', 'team1_blk', 'team1_tov',
               'team2_w_pct', 'team2_fgm', 'team2_fg_pct',
       'team2_fg3_pct', 'team2_ftm', 'team2_fta', 'team2_ft_pct', 'team2_reb',
       'team2_ast', 'team2_stl', 'team2_blk', 'team2_tov']
predict

,team1_matchup,team1_team_id_x,team1_w_pct,team1_fgm,team1_fg_pct,team1_fg3_pct,team1_fta,team1_ft_pct,team1_reb,team1_ast,...,team2_fta,team2_ft_pct,team2_reb,team2_ast,team2_stl,team2_blk,team2_tov,book_id,homeTeamPrice,awayTeamPrice
0,CLE vs. GSW,1.610613e+09,1.0,39.0,0.453,0.368,16.0,1.0,44.0,25.0,...,16.0,1.0,44.0,25.0,7.0,13.0,8.0,19.0,6725.0,-11450.0
1,CLE vs. GSW,1.610613e+09,1.0,39.0,0.453,0.368,16.0,1.0,44.0,25.0,...,37.0,1.0,39.0,20.0,6.0,5.0,12.0,19.0,-145.0,125.0
2,CLE vs. GSW,1.610613e+09,1.0,39.0,0.453,0.368,16.0,1.0,44.0,25.0,...,37.0,1.0,39.0,20.0,6.0,5.0,12.0,999996.0,-145.0,125.0
3,CLE vs. GSW,1.610613e+09,1.0,39.0,0.453,0.368,16.0,1.0,44.0,25.0,...,16.0,1.0,39.0,15.0,5.0,7.0,11.0,19.0,-145.0,125.0
4,CLE vs. GSW,1.610613e+09,1.0,39.0,0.453,0.368,16.0,1.0,44.0,25.0,...,16.0,1.0,39.0,15.0,5.0,7.0,11.0,999996.0,-145.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127682,PHI vs. ATL,1.610613e+09,0.0,26.0,0.333,0.235,22.0,1.0,42.0,11.0,...,24.0,1.0,27.0,21.0,7.0,0.0,15.0,93.0,100.0,-120.0
127683,PHI vs. ATL,1.610613e+09,0.0,26.0,0.333,0.235,22.0,1.0,42.0,11.0,...,25.0,1.0,47.0,19.0,10.0,6.0,16.0,19.0,-245.0,205.0
127684,PHI vs. ATL,1.610613e+09,0.0,26.0,0.333,0.235,22.0,1.0,42.0,11.0,...,25.0,1.0,47.0,19.0,10.0,6.0,16.0,999996.0,-240.0,200.0
127685,PHI vs. ATL,1.610613e+09,0.0,26.0,0.333,0.235,22.0,1.0,42.0,11.0,...,22.0,1.0,42.0,11.0,9.0,4.0,18.0,19.0,-245.0,205.0
